In [1]:
!pip3 install praw

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 6.0 MB/s eta 0:00:00


In [112]:
import pandas as pd
from datetime import datetime, timezone

In [19]:
# original data
df = pd.read_csv("./dataset/reddit_wsb.csv")

In [110]:
df_new = pd.DataFrame(columns = df.columns)

In [11]:
import praw

reddit = praw.Reddit(
    client_id="*****",
    client_secret="*****",
    user_agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
)

In [70]:
subreddit = reddit.subreddit("wallstreetbets")

In [113]:
limit = 2000
for g in [subreddit.new(limit=limit), subreddit.top(limit=limit), subreddit.hot(limit=limit), subreddit.rising(limit=limit)]:
    for submission in g:
        data = {
            "title": submission.title,
            "score": submission.score,
            "id": submission.id,
            "url": submission.url,
            "comms_num": submission.num_comments,
            "created": submission.created_utc,
            "body": submission.selftext,
            "timestamp": datetime.fromtimestamp(submission.created_utc, timezone.utc).strftime('%Y-%m-%d %H:%M:%S')
        }
        # df_new.concat(data, ignore_index = True)
        df_new.loc[len(df_new)] = data

In [94]:
reddit.auth.limits

{'remaining': 576.0, 'reset_timestamp': 1701886799.027841, 'used': 24}

In [127]:
df_new = df_new.drop_duplicates(["id"], ignore_index = True)
# find the data for 2023
df_new = df_new[df_new["created"] >= datetime.timestamp(datetime(2023, 1, 1, tzinfo=timezone.utc))].reset_index(drop = True)
df_new = df_new.sort_values(["created"], ignore_index = True)
df_new.to_csv("./dataset/reddit_wsb_2023.csv", index = False)